# import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.1.0


# Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & pre-processing function

In [3]:

def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

# Performance function

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)    
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

# Model function

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def batch_norm():
    return tf.keras.layers.BatchNormalization()

# Create model(class)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class,self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()
        
        self.model = tf.keras.Sequential()
        self.model.add(flatten())
        
        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(batch_norm())
            self.model.add(relu())
            
        self.model.add(dense(label_dmi, weight_init))
        
    def call(self, x, training=None, maks=None):
        x = self.model(x)
        return x

# Create model(function)

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4):
        model.add(dense(512, weight_init))
        model.add(batch_norm())
        model.add(relu())
        

    model.add(dense(label_dim, weight_init))

    return model

# Define data & hyper parameter

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

# Define model & optimizer & writer

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_batchnorm'


checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

# Restore checkpoint & start train or test phase

In [10]:

if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.9461, train_loss: 0.37046313, train_accuracy: 0.3359, test_Accuracy: 0.2563
Epoch: [ 0] [    1/  468] time: 1.0622, train_loss: 0.36707956, train_accuracy: 0.2578, test_Accuracy: 0.2736
Epoch: [ 0] [    2/  468] time: 1.1742, train_loss: 0.36316729, train_accuracy: 0.3516, test_Accuracy: 0.3365
Epoch: [ 0] [    3/  468] time: 1.2852, train_loss: 0.28742915, train_accuracy: 0.4531, test_Accuracy: 0.4273
Epoch: [ 0] [    4/  468] time: 1.3993, train_loss: 0.26367617, train_accuracy: 0.5625, test_Accuracy: 0.5323
Epoch: [ 0] [    5/  468] time: 1.5203, train_loss: 0.27595186, train_accuracy: 0.6094, test_Accuracy: 0.6218
Epoch: [ 0] [    6/  468] time: 1.6323, train_loss: 0.27553353, train_accuracy: 0.6797, test_Accuracy: 0.6682
Epoch: [ 0] [    7/  468] time: 1.7453, train_loss: 0.30800280, train_accuracy: 0.6797, test_Accuracy: 0.7102
Epoch: [ 0] [    8/  468] time: 1.8553,

Epoch: [ 0] [   13/  468] time: 2.4083, train_loss: 0.28289825, train_accuracy: 0.7812, test_Accuracy: 0.7858
Epoch: [ 0] [   14/  468] time: 2.5223, train_loss: 0.28538850, train_accuracy: 0.8125, test_Accuracy: 0.7992
Epoch: [ 0] [   15/  468] time: 2.6353, train_loss: 0.22249934, train_accuracy: 0.7969, test_Accuracy: 0.8138
Epoch: [ 0] [   16/  468] time: 2.7483, train_loss: 0.16946229, train_accuracy: 0.8672, test_Accuracy: 0.8259
Epoch: [ 0] [   17/  468] time: 2.8613, train_loss: 0.18268856, train_accuracy: 0.8438, test_Accuracy: 0.8333
Epoch: [ 0] [   18/  468] time: 2.9763, train_loss: 0.16519532, train_accuracy: 0.8750, test_Accuracy: 0.8427
Epoch: [ 0] [   19/  468] time: 3.0913, train_loss: 0.14938143, train_accuracy: 0.8906, test_Accuracy: 0.8499
Epoch: [ 0] [   20/  468] time: 3.2063, train_loss: 0.25567544, train_accuracy: 0.8359, test_Accuracy: 0.8602
Epoch: [ 0] [   21/  468] time: 3.3203, train_loss: 0.16633557, train_accuracy: 0.8750, test_Accuracy: 0.8686
Epoch: [ 0

Epoch: [ 0] [   27/  468] time: 4.0268, train_loss: 0.12392959, train_accuracy: 0.9297, test_Accuracy: 0.8910
Epoch: [ 0] [   28/  468] time: 4.1389, train_loss: 0.19169213, train_accuracy: 0.9062, test_Accuracy: 0.8907
Epoch: [ 0] [   29/  468] time: 4.2539, train_loss: 0.21463622, train_accuracy: 0.9219, test_Accuracy: 0.8920
Epoch: [ 0] [   30/  468] time: 4.3689, train_loss: 0.20917796, train_accuracy: 0.8672, test_Accuracy: 0.8919
Epoch: [ 0] [   31/  468] time: 4.4799, train_loss: 0.32203662, train_accuracy: 0.8281, test_Accuracy: 0.8939
Epoch: [ 0] [   32/  468] time: 4.5909, train_loss: 0.23588362, train_accuracy: 0.8594, test_Accuracy: 0.8971
Epoch: [ 0] [   33/  468] time: 4.7039, train_loss: 0.20150131, train_accuracy: 0.9141, test_Accuracy: 0.8962
Epoch: [ 0] [   34/  468] time: 4.8154, train_loss: 0.15335810, train_accuracy: 0.9062, test_Accuracy: 0.8992
Epoch: [ 0] [   35/  468] time: 4.9344, train_loss: 0.25230429, train_accuracy: 0.8828, test_Accuracy: 0.9019
Epoch: [ 0

Epoch: [ 0] [   41/  468] time: 5.6194, train_loss: 0.16508704, train_accuracy: 0.9297, test_Accuracy: 0.9034
Epoch: [ 0] [   42/  468] time: 5.7324, train_loss: 0.17306747, train_accuracy: 0.9375, test_Accuracy: 0.9044
Epoch: [ 0] [   43/  468] time: 5.8504, train_loss: 0.21620888, train_accuracy: 0.9219, test_Accuracy: 0.9067
Epoch: [ 0] [   44/  468] time: 5.9674, train_loss: 0.15893054, train_accuracy: 0.8984, test_Accuracy: 0.9127
Epoch: [ 0] [   45/  468] time: 6.0784, train_loss: 0.23367418, train_accuracy: 0.9219, test_Accuracy: 0.9161
Epoch: [ 0] [   46/  468] time: 6.1874, train_loss: 0.20637286, train_accuracy: 0.9141, test_Accuracy: 0.9202
Epoch: [ 0] [   47/  468] time: 6.3072, train_loss: 0.15810063, train_accuracy: 0.9219, test_Accuracy: 0.9215
Epoch: [ 0] [   48/  468] time: 6.4272, train_loss: 0.17841493, train_accuracy: 0.9297, test_Accuracy: 0.9201
Epoch: [ 0] [   49/  468] time: 6.5472, train_loss: 0.08235097, train_accuracy: 0.9766, test_Accuracy: 0.9197
Epoch: [ 0

Epoch: [ 0] [   55/  468] time: 7.2372, train_loss: 0.10438251, train_accuracy: 0.9844, test_Accuracy: 0.9259
Epoch: [ 0] [   56/  468] time: 7.3542, train_loss: 0.06940019, train_accuracy: 0.9766, test_Accuracy: 0.9278
Epoch: [ 0] [   57/  468] time: 7.4682, train_loss: 0.23827548, train_accuracy: 0.8906, test_Accuracy: 0.9292
Epoch: [ 0] [   58/  468] time: 7.5812, train_loss: 0.08673830, train_accuracy: 0.9531, test_Accuracy: 0.9303
Epoch: [ 0] [   59/  468] time: 7.6932, train_loss: 0.20305753, train_accuracy: 0.9141, test_Accuracy: 0.9278
Epoch: [ 0] [   60/  468] time: 7.8022, train_loss: 0.15221649, train_accuracy: 0.9297, test_Accuracy: 0.9272
Epoch: [ 0] [   61/  468] time: 7.9202, train_loss: 0.07893082, train_accuracy: 0.9531, test_Accuracy: 0.9260
Epoch: [ 0] [   62/  468] time: 8.0392, train_loss: 0.15646553, train_accuracy: 0.9297, test_Accuracy: 0.9254
Epoch: [ 0] [   63/  468] time: 8.1512, train_loss: 0.17333126, train_accuracy: 0.9453, test_Accuracy: 0.9247
Epoch: [ 0

...

Epoch: [ 0] [  405/  468] time: 47.6307, train_loss: 0.05690374, train_accuracy: 0.9844, test_Accuracy: 0.9702
Epoch: [ 0] [  406/  468] time: 47.7397, train_loss: 0.08991600, train_accuracy: 0.9609, test_Accuracy: 0.9697
Epoch: [ 0] [  407/  468] time: 47.8497, train_loss: 0.13039467, train_accuracy: 0.9688, test_Accuracy: 0.9698
Epoch: [ 0] [  408/  468] time: 47.9647, train_loss: 0.07446085, train_accuracy: 0.9766, test_Accuracy: 0.9698
Epoch: [ 0] [  409/  468] time: 48.0817, train_loss: 0.10244007, train_accuracy: 0.9531, test_Accuracy: 0.9705
Epoch: [ 0] [  410/  468] time: 48.1937, train_loss: 0.05272925, train_accuracy: 0.9922, test_Accuracy: 0.9704
Epoch: [ 0] [  411/  468] time: 48.3047, train_loss: 0.08624244, train_accuracy: 0.9766, test_Accuracy: 0.9712
Epoch: [ 0] [  412/  468] time: 48.4207, train_loss: 0.09410095, train_accuracy: 0.9922, test_Accuracy: 0.9719
Epoch: [ 0] [  413/  468] time: 48.5317, train_loss: 0.03594018, train_accuracy: 0.9922, test_Accuracy: 0.9714
E

Epoch: [ 0] [  419/  468] time: 49.2020, train_loss: 0.04444512, train_accuracy: 0.9844, test_Accuracy: 0.9697
Epoch: [ 0] [  420/  468] time: 49.3160, train_loss: 0.03488861, train_accuracy: 0.9844, test_Accuracy: 0.9683
Epoch: [ 0] [  421/  468] time: 49.4280, train_loss: 0.09606607, train_accuracy: 0.9766, test_Accuracy: 0.9680
Epoch: [ 0] [  422/  468] time: 49.5400, train_loss: 0.02448475, train_accuracy: 1.0000, test_Accuracy: 0.9681
Epoch: [ 0] [  423/  468] time: 49.6500, train_loss: 0.03447254, train_accuracy: 0.9766, test_Accuracy: 0.9684
Epoch: [ 0] [  424/  468] time: 49.7670, train_loss: 0.07007386, train_accuracy: 0.9844, test_Accuracy: 0.9684
Epoch: [ 0] [  425/  468] time: 49.8780, train_loss: 0.07207786, train_accuracy: 0.9766, test_Accuracy: 0.9677
Epoch: [ 0] [  426/  468] time: 49.9900, train_loss: 0.03252304, train_accuracy: 1.0000, test_Accuracy: 0.9669
Epoch: [ 0] [  427/  468] time: 50.1087, train_loss: 0.03105606, train_accuracy: 0.9844, test_Accuracy: 0.9667
E

Epoch: [ 0] [  433/  468] time: 50.7828, train_loss: 0.04403586, train_accuracy: 0.9922, test_Accuracy: 0.9669
Epoch: [ 0] [  434/  468] time: 50.8928, train_loss: 0.03655367, train_accuracy: 0.9844, test_Accuracy: 0.9674
Epoch: [ 0] [  435/  468] time: 51.0098, train_loss: 0.06483948, train_accuracy: 0.9609, test_Accuracy: 0.9672
Epoch: [ 0] [  436/  468] time: 51.1208, train_loss: 0.08623519, train_accuracy: 0.9844, test_Accuracy: 0.9674
Epoch: [ 0] [  437/  468] time: 51.2298, train_loss: 0.14277558, train_accuracy: 0.9766, test_Accuracy: 0.9689
Epoch: [ 0] [  438/  468] time: 51.3428, train_loss: 0.04686233, train_accuracy: 0.9688, test_Accuracy: 0.9697
Epoch: [ 0] [  439/  468] time: 51.4598, train_loss: 0.06766666, train_accuracy: 0.9922, test_Accuracy: 0.9703
Epoch: [ 0] [  440/  468] time: 51.5718, train_loss: 0.04824049, train_accuracy: 0.9766, test_Accuracy: 0.9706
Epoch: [ 0] [  441/  468] time: 51.6818, train_loss: 0.04409048, train_accuracy: 0.9844, test_Accuracy: 0.9696
E

Epoch: [ 0] [  447/  468] time: 52.3558, train_loss: 0.04438561, train_accuracy: 0.9609, test_Accuracy: 0.9717
Epoch: [ 0] [  448/  468] time: 52.4648, train_loss: 0.04155465, train_accuracy: 0.9922, test_Accuracy: 0.9712
Epoch: [ 0] [  449/  468] time: 52.5838, train_loss: 0.03227327, train_accuracy: 1.0000, test_Accuracy: 0.9702
Epoch: [ 0] [  450/  468] time: 52.7008, train_loss: 0.05052827, train_accuracy: 0.9922, test_Accuracy: 0.9690
Epoch: [ 0] [  451/  468] time: 52.8158, train_loss: 0.06389858, train_accuracy: 0.9766, test_Accuracy: 0.9684
Epoch: [ 0] [  452/  468] time: 52.9258, train_loss: 0.09852162, train_accuracy: 0.9688, test_Accuracy: 0.9667
Epoch: [ 0] [  453/  468] time: 53.0418, train_loss: 0.03723211, train_accuracy: 0.9922, test_Accuracy: 0.9661
Epoch: [ 0] [  454/  468] time: 53.1611, train_loss: 0.07521586, train_accuracy: 0.9609, test_Accuracy: 0.9656
Epoch: [ 0] [  455/  468] time: 53.2711, train_loss: 0.12701899, train_accuracy: 0.9688, test_Accuracy: 0.9644
E

Epoch: [ 0] [  461/  468] time: 53.9712, train_loss: 0.07751955, train_accuracy: 0.9844, test_Accuracy: 0.9670
Epoch: [ 0] [  462/  468] time: 54.0802, train_loss: 0.06553221, train_accuracy: 0.9922, test_Accuracy: 0.9656
Epoch: [ 0] [  463/  468] time: 54.1982, train_loss: 0.07174334, train_accuracy: 0.9766, test_Accuracy: 0.9646
Epoch: [ 0] [  464/  468] time: 54.3142, train_loss: 0.14209354, train_accuracy: 0.9609, test_Accuracy: 0.9631
Epoch: [ 0] [  465/  468] time: 54.4302, train_loss: 0.07035327, train_accuracy: 0.9609, test_Accuracy: 0.9621
Epoch: [ 0] [  466/  468] time: 54.5492, train_loss: 0.11128121, train_accuracy: 0.9531, test_Accuracy: 0.9597
Epoch: [ 0] [  467/  468] time: 54.6642, train_loss: 0.10695932, train_accuracy: 0.9609, test_Accuracy: 0.9595


# Test Accuracy : 96.34%